# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

In [2]:
import pandas as pd

In [723]:
import numpy as np

In [619]:
toxic = pd.read_csv('labeled.csv')

In [620]:
toxic.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [657]:
train, test = train_test_split(toxic, test_size=0.2, shuffle=True, random_state=255)
train.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)

#### Векторайзер с дефолтной токенизацией

In [210]:
vectorizer = TfidfVectorizer(min_df=5, max_df=0.2)
X = vectorizer.fit_transform(train.comment)

In [211]:
X.shape

(11529, 6707)

In [212]:
X_test = vectorizer.transform(test.comment)

In [213]:
y = train.toxic.values
y_test = test.toxic.values

In [214]:
clf = DecisionTreeClassifier(max_depth=8, class_weight='balanced')
clf.fit(X, y)
preds = clf.predict(X_test)

print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.87      0.33      0.48      1925
         1.0       0.40      0.90      0.56       958

    accuracy                           0.52      2883
   macro avg       0.64      0.62      0.52      2883
weighted avg       0.72      0.52      0.50      2883



#### Векторайзер с токенизацией из razdel

In [215]:
!pip install razdel

In [216]:
from razdel import tokenize

In [217]:
def my_tokenizer(text):
    tokens = list(tokenize(text))
    return [_.text for _ in tokens]

In [218]:
vectorizer_razdel = TfidfVectorizer(min_df=5, max_df=0.2, tokenizer=my_tokenizer)
X_1 = vectorizer_razdel.fit_transform(train.comment)

In [219]:
X_1.shape

(11529, 6756)

In [220]:
X_test_1 = vectorizer_razdel.transform(test.comment)

In [221]:
y_1 = train.toxic.values
y_test_1 = test.toxic.values

In [222]:
clf_razdel = DecisionTreeClassifier(max_depth=8, class_weight='balanced')
clf_razdel.fit(X_1, y_1)
preds = clf_razdel.predict(X_test_1)

print(classification_report(y_test_1, preds))

              precision    recall  f1-score   support

         0.0       0.87      0.42      0.56      1925
         1.0       0.43      0.87      0.57       958

    accuracy                           0.57      2883
   macro avg       0.65      0.64      0.57      2883
weighted avg       0.72      0.57      0.57      2883



Качество классификатора с кастомным токенайзером из razdel выше, конкурировать может только показатель полноты по классам, но overall метрики очевидно возросли (но не сильно)

## Задание 2 (3 балла)

Преобразуйте таблицу с абсолютными частотностями в семинарской тетрадке в таблицу с tfidf значениями. (Таблица - https://i.ibb.co/r5Nc2HC/abs-bow.jpg) Формула tfidf есть в семинаре на картнике с пояснениями на английском. 
Считать нужно в питоне. Формат итоговой таблицы может быть любым, главное, чтобы был код и можно было воспроизвести вычисления. 

In [187]:
import math

In [188]:
data = pd.read_csv('tfidf_sample.txt', sep='\t', names=['я', 'ты', 'и', 'только', 'не', 'он'], header=0, index_col=0)
data.head()

,я,ты,и,только,не,он
я и ты,1,1,1,0,0,0
ты и я,1,1,1,0,0,0
"я, я и только я",3,0,1,1,0,0
только не я,1,0,0,1,1,0
он,0,0,0,0,0,1


In [189]:
N = len(data)

In [190]:
def count_tfidf(word, num_row):
    tf = data.loc[num_row][word] / sum(data.loc[num_row])
    df = sum(data[word] > 0)
    return tf * math.log(N / df)

In [191]:
data_new = pd.DataFrame(columns=data.columns)

In [192]:
for row in data.iterrows():
    one_line = []
    for title in row[1].keys():
        one_line.append(count_tfidf(title, row[0]))
    data_new.loc[row[0]] = one_line

In [193]:
data_new

,я,ты,и,только,не,он
я и ты,0.074381,0.30543,0.170275,0.000000,0.000000,0.000000
ты и я,0.074381,0.30543,0.170275,0.000000,0.000000,0.000000
"я, я и только я",0.133886,0.00000,0.102165,0.183258,0.000000,0.000000
только не я,0.074381,0.00000,0.000000,0.305430,0.536479,0.000000
он,0.000000,0.00000,0.000000,0.000000,0.000000,1.609438


## Задание 3 (2 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к классификаторам:   
а) один должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров  
в) у классификатора должно быть задано вручную как минимум 2 параметра  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

In [659]:
train.head()

,comment,toxic
0,Первый раз вижу чтобы в рецепт блинов кто-то д...,0.0
1,"А если на конкретных людей в суд подавать,а не...",0.0
2,"Железнодорожники вообще закрытая каста, госуда...",0.0
3,"А, вот, нагуглилось - видосик сверху Ну да, вы...",0.0
4,Лучше что-то среднее между двумя частями. Перв...,0.0


`train, test = train_test_split(toxic, test_size=0.2, shuffle=True)
train.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)`

In [331]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\katri\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [333]:
from nltk.corpus import stopwords

#### Классификатор 1

In [664]:
train.shape, test.shape

((11529, 2), (2883, 2))

In [841]:
# stopwords из nltk и токенайзер из razdel снижали качество
# все параметры пробовала менять, если стоит дефолтный - значит с кастомным становилось хуже
# Например, ngram_range=(1, 2)/(2, 2), analyzer='char'/'char_wb', lowercase=False
count_vectorizer = CountVectorizer(lowercase=True,
                             max_df=0.1,
                             min_df=3,
                             ngram_range=(1, 1),
                             analyzer='word',
                            stop_words=stopwords.words('russian'))
Xc = count_vectorizer.fit_transform(train.comment)

In [842]:
Xc_test = count_vectorizer.transform(test.comment)

In [843]:
yc = train.toxic.values
yc_test = test.toxic.values

In [844]:
clf_NB = MultinomialNB(alpha=1., fit_prior=True)
clf_NB.fit(Xc, yc)
preds = clf_NB.predict(Xc_test)

print(classification_report(yc_test, preds))

              precision    recall  f1-score   support

         0.0       0.87      0.92      0.90      1925
         1.0       0.82      0.73      0.77       958

    accuracy                           0.86      2883
   macro avg       0.85      0.83      0.83      2883
weighted avg       0.86      0.86      0.86      2883



In [845]:
probas_NB = clf_NB.predict_proba(Xc_test)

In [846]:
top10 = test.iloc[np.argsort(probas_NB[:, 1])[-10:]]
top10

,comment,toxic
774,Могли абонемент к шлюхе гораздо лучшего качест...,1.0
480,"Все согласны, блядь. Всем по полному ебучему з...",0.0
2289,Ну так я же не предлагаю перекатываться на дру...,0.0
2658,"Ебать, кто смотрит сыромятникова?? Где еще мож...",1.0
709,"Стас, никому, кроме тебя и армии твоих подсосо...",1.0
1883,"Стас, никому, кроме тебя и армии твоих подсосо...",1.0
2507,"Но Трамп и в правда дегенерат. Жаль, что этот ...",1.0
2455,"ты, блять, троллишь чтоли или просто перепутал...",1.0
1551,"Блеаадь, как же обидно когда создаешь тред, па...",1.0
801,АЛЛО БЛЯДЬ АБУ ТЫ Ч ОХУЕЛ? ВЫГОНЯЙ НАХУЙ СВОЕГ...,1.0


In [847]:
for i in top10[top10['toxic'] == 0]['comment']:
    print(i)

Все согласны, блядь. Всем по полному ебучему завтраку, подводит итог Бегби. Рентон не верит своим ушам. Ему хочется послать Бегби на хуй. Но он перебарывает этот порыв и только медленно качает головой: Я не ем мяса, Франко. баное вегетарианство. баный пиздёж. Ты должен есть мясо. баный торчок заботится, на хуй, о том, что он вводит в свой в организм! Я угораю, бля! Просто я не люблю мяса, отвечает Рентон, чувствуя себя полным дураком, и все начинают хихикать. Только не пизди мне о том, что тебе жалко ёбаных животных. Вспомни тех ёбаных собак и кошек, в которых мы с тобой, на хуй, стреляли из духовых ружбаек! А ещё ёбаных голубей, которых мы жгли живьём. Этот чувак делал ебучие шутихи типа как фейерверки из белых мышей. Мне не жалко животных. Просто я не могу их есть, Рентон пожимает плечами, смущённый тем, что Келли узнала о его подростковых зверствах. Бессердечные ублюдки. Не представляю, как можно выстрелить в собаку, ехидничает Элисон, качая головой. А я не представляю себе, как мож

Результат неплохой, оба комментария вообще довольно агрессивные, в них много мата, окрашенных слов, разных оскорбительных модификаций

Просто посмотреть на важность признаков:

In [849]:
print(np.take(count_vectorizer.get_feature_names(), clf_NB.feature_log_prob_[1, :].argsort()[::-1][:10]))

['просто' 'тебе' 'всё' 'вообще' 'почему' 'ещё' 'хохлы' 'нахуй' 'лет'
 'хохлов']


#### Классификатор 2

In [903]:
tfidf_vectorizer = TfidfVectorizer(lowercase=True,
                                   max_df=0.2,
                                   min_df=2,
                                   analyzer='word',
                                   tokenizer=my_tokenizer)
Xt = tfidf_vectorizer.fit_transform(train.comment)

In [904]:
Xt_test = tfidf_vectorizer.transform(test.comment)

In [905]:
yt = train.toxic.values
yt_test = test.toxic.values

In [906]:
clf_KN = KNeighborsClassifier(n_neighbors=15, weights='distance', metric='cosine')
clf_KN.fit(Xt, yt)
preds = clf_KN.predict(Xt_test)

print(classification_report(yt_test, preds))

              precision    recall  f1-score   support

         0.0       0.87      0.89      0.88      1925
         1.0       0.76      0.73      0.75       958

    accuracy                           0.83      2883
   macro avg       0.82      0.81      0.81      2883
weighted avg       0.83      0.83      0.83      2883



In [907]:
probas_KN = clf_KN.predict_proba(Xt_test)

In [908]:
top10 = test.iloc[np.argsort(probas_KN[:, 1])[-10:]]
top10

,comment,toxic
2114,"Теперь хохлов бкдкт травить анашой, чтобы не п...",1.0
1370,Смерть достойная хохла\n,1.0
1383,"скажите, а что заставляет пользоваться подобны...",0.0
2052,У хохлов вообще сказок нет. Ну сказка про дене...,1.0
2608,"Я Курицына никогда не ебал, но в этот раз точн...",1.0
98,Весело ломать людям жизни ? В такие моменты я ...,1.0
2629,"Президент США Дональд Трамп заявил, что всерье...",0.0
2635,Ебать бомбит прокладки\n,1.0
1068,"Перепутали в роддоме негритенка, хохла и русск...",1.0
1228,Гандоны - это ты и твой хозяин - пидорас-приго...,1.0


In [909]:
for i in top10[top10['toxic'] == 0]['comment']:
    print(i)

скажите, а что заставляет пользоваться подобными мемами фекально-дендроидной архитектуры?

Президент США Дональд Трамп заявил, что всерьез рассматривает возможность членства Бразилии в НАТО. Об этом он рассказал на совместной с бразильским коллегой Жаиром Болсонару пресс-конференции, передает Reuters. Вашингтон намерен объявить Бразилию главным стратегическим союзником среди государств, не входящих в НАТО, и допускает возможность членства Бразилии в альянсе. Это значительно укрепит безопасность и сотрудничество между нашими странами , объяснил американский лидер. Кроме того, Трамп поддерживает желание Болсонару присоединиться к Организации экономического сотрудничества и развития. По словам американского лидера, отношения между странами никогда не были настолько близкими . Ранее Болсонару призвал очистить Бразилию от грязных левых взглядов . Отмечалось, что поездка бразильского лидера в Вашингтон должна продемонстрировать приверженность южноамериканской страны США, что является отклоне

Сложный случай. Во втором отзые совсем не могу понять, откуда токсичность (только если тематика политики опять же). В первлм всё-таки есть предположения, но тоже вряд ли модель опиралась на эту лексику.

## *Задание 4 (2 балла)

Для классификаторов LogisticRegression, Decision Trees, Naive Bayes, Random Forest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.

In [817]:
count_vectorizer = CountVectorizer(stop_words=stopwords.words('russian') + ["это"], max_df=0.5)

In [818]:
X = count_vectorizer.fit_transform(train.comment)

In [819]:
X_test = count_vectorizer.transform(test.comment)

In [820]:
y = train.toxic.values
y_test = test.toxic.values

#### Логистическая регрессия

In [821]:
clf_LR = LogisticRegression(C=0.1, class_weight='balanced')

In [822]:
clf_LR.fit(X, y)

LogisticRegression(C=0.1, class_weight='balanced')

In [823]:
clf_LR.coef_

array([[ 0.50383534,  0.33069627, -0.03772816, ..., -0.04800185,
        -0.0134363 , -0.00685607]])

In [824]:
print(np.take(count_vectorizer.get_feature_names(), clf_LR.coef_.argsort()[0][-10:]))

['лол' 'тупой' 'сука' 'нахуй' 'пиздец' 'дебил' 'блядь' 'тебе' 'хохлов'
 'хохлы']


#### Decision Trees

In [825]:
clf_trees = DecisionTreeClassifier(max_depth=8, class_weight='balanced')
clf_trees.fit(X, y)

DecisionTreeClassifier(class_weight='balanced', max_depth=8)

In [826]:
clf_trees.feature_importances_

array([0., 0., 0., ..., 0., 0., 0.])

In [827]:
print(np.take(count_vectorizer.get_feature_names(), clf_trees.feature_importances_.argsort()[-10:]))

['фотографии' 'говорит' 'блядь' 'года' 'например' 'нахуй' 'хохлов' 'очень'
 'тебе' 'хохлы']


#### Naive Bayes

In [828]:
clf_NB = MultinomialNB(alpha=1., fit_prior=True)
clf_NB.fit(X, y)

MultinomialNB()

In [829]:
pos_class = clf_NB.feature_log_prob_[1, :].argsort()[::-1]

In [830]:
print(np.take(count_vectorizer.get_feature_names(), pos_class[:10]))

['просто' 'тебе' 'всё' 'вообще' 'почему' 'ещё' 'хохлы' 'нахуй' 'лет'
 'хохлов']


#### Random Forest

In [831]:
clf_rf = RandomForestClassifier(n_estimators=100, max_depth=20)
clf_rf.fit(X, y)

RandomForestClassifier(max_depth=20)

In [832]:
print(np.take(count_vectorizer.get_feature_names(), clf_rf.feature_importances_.argsort()[-10:]))

['лол' 'русских' 'похуй' 'пиздец' 'тупые' 'тупой' 'хохлов' 'блядь' 'тебе'
 'хохлы']
